In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

df = pd.read_excel("C:\\Users\\Admin\\Desktop\\DATN\\data.xlsx")
df


,ResponseId,MainBranch,Age,Country,EdLevel,WorkExp,CompTotal,RemoteWork,Employment,OrgSize,...,OfficeStackSyncHaveWorkedWith,SOComm,AISelect,AISent,AIBen,AIAcc,AIToolCurrently Using,TBranch,ICorPM,BranchGroup
0,393,I am a developer by profession,35-44,United Kingdom Of Great Britain And Northern I...,Professional,18,126420.0,Hybrid,Employ,Large,...,Slack,No,Yes,Unfavorable,Improve accuracy in coding,Somewhat distrust,Writing code,Yes,People manager,Developer
1,417,I am a developer by profession,35-44,Brazil,Master,17,170000.0,Remote,Employ-Freelance,Medium,...,Google Meet-Telegram-Whatsapp-Zoom,No,Yes,Favorable,Increase productivity,Somewhat distrust,Writing code-Documenting code-Testing code,Yes,Individual contributor,Developer
2,424,I am a developer by profession,25-34,Italy,No Degree,4,62000.0,Remote,Employ-Freelance,Medium,...,Google Meet-Mattermost-Slack,Yes,Yes,Unfavorable,Increase productivity,Highly distrust,Writing code,Yes,Individual contributor,Developer
3,444,"I am not primarily a developer, but I write co...",35-44,United Kingdom Of Great Britain And Northern I...,Bachelor,20,127000.0,Remote,Employ,Enterprise,...,Google Meet-Microsoft Teams-Skype-Slack-Telegr...,Somewhat,Yes,Favorable,Increase productivity-Greater efficiency-Speed...,Somewhat trust,Writing code-Search for answers-Generating con...,Yes,People manager,Semi-technical
4,445,I am a developer by profession,25-34,United Kingdom Of Great Britain And Northern I...,Bachelor,10,115000.0,Remote,Employ-Student,Small,...,Slack-Zoom,Neutral,Yes,Favorable,Speed up learning,Neither trust nor distrust,Learning about a codebase-Writing code,Yes,Individual contributor,Developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,65106,I am a developer by profession,25-34,United States Of America,Bachelor,8,155000.0,Hybrid,Employ,Enterprise,...,Symphony-Zoom,Somewhat,Yes,Favorable,Increase productivity-Greater efficiency,Somewhat trust,Writing code-Debugging and getting help-Testin...,Yes,Individual contributor,Developer
10101,65152,I am a developer by profession,35-44,Estonia,Master,14,90000.0,Hybrid,Employ,Large,...,Slack-Zoom,Neutral,Yes,Favorable,Increase productivity-Greater efficiency,Somewhat distrust,Writing code-Generating content or synthetic data,Yes,Individual contributor,Developer
10102,65164,I am a developer by profession,18-24,Germany,Associate,3,46000.0,Hybrid,Employ,Small,...,Discord-Microsoft Teams-Telegram-Whatsapp,Neutral,Yes,Favorable,Increase productivity-Greater efficiency-Impro...,Somewhat trust,Writing code-Documenting code-Debugging and ge...,Yes,Individual contributor,Developer
10103,65167,I am a developer by profession,25-34,Spain,Bachelor,8,81600.0,Remote,Employ,Large,...,Google Chat-Google Meet-Microsoft Teams-Skype-...,No,Yes,Very favorable,Increase productivity-Greater efficiency-Speed...,Somewhat trust,Learning about a codebase-Debugging and gettin...,Yes,Individual contributor,Developer


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10105 entries, 0 to 10104
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      10105 non-null  int64  
 1   MainBranch                      10105 non-null  object 
 2   Age                             10105 non-null  object 
 3   Country                         10105 non-null  object 
 4   EdLevel                         10105 non-null  object 
 5   WorkExp                         10105 non-null  int64  
 6   CompTotal                       10105 non-null  float64
 7   RemoteWork                      10105 non-null  object 
 8   Employment                      10105 non-null  object 
 9   OrgSize                         10105 non-null  object 
 10  JobSat                          10105 non-null  int64  
 11  TimeSearching                   10105 non-null  int64  
 12  TimeAnswering                   

In [4]:
class Modeling_Thunhap:
    def __init__(self, df):
        self.df = df.copy()
        self.pipeline = None
        self.df_model = None

    def chuyen_doi_workexp(self, val):
        if val == 'More than 50 years':
              return 51
        elif val == 'Less than 1 year':
            return 0.5
        try:
            return float(val)
        except:
            return None

    def tien_xu_ly(self):
        self.df_model = self.df[['DevType', 'Country', 'EdLevel', 'WorkExp', 'CompTotal', 'RemoteWork', 'OrgSize']].copy()
        self.df_model['WorkExp'] = self.df_model['WorkExp'].apply(self.chuyen_doi_workexp)
        self.df_model = self.df_model.dropna(subset=['WorkExp', 'CompTotal'])
        self.df_model = self.df_model[(self.df_model['CompTotal'] > 1000) & 
                                      (self.df_model['CompTotal'] < 5e21)]

        # Lọc quốc gia có ít nhất 30 mẫu
        valid_countries = self.df_model['Country'].value_counts()
        valid_countries = valid_countries[valid_countries >= 30].index
        self.df_model = self.df_model[self.df_model['Country'].isin(valid_countries)]

    def train_model(self):
        X = self.df_model.drop(columns='CompTotal')
        y = self.df_model['CompTotal']

        cot_so = ['WorkExp']
        cot_phan_loai = ['DevType', 'Country', 'EdLevel', 'RemoteWork']

        xu_ly = ColumnTransformer([
            ('so', SimpleImputer(strategy='mean'), cot_so),
            ('phan_loai', Pipeline([
                ('impute', SimpleImputer(strategy='most_frequent')),
                ('encode', OneHotEncoder(handle_unknown='ignore'))
            ]), cot_phan_loai)
        ])

        self.pipeline = Pipeline([
            ('xu_ly', xu_ly),
            ('model', LinearRegression())
        ])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.pipeline.fit(X_train, y_train)

        self.df_model['LuongDuDoan'] = self.pipeline.predict(X)
        self.df_model = self.df_model[(self.df_model['LuongDuDoan'] > 1000) & 
                                      (self.df_model['LuongDuDoan'] < 5e21)]

    def tong_hop(self, cot_nhom, ten_moi):
        kq = self.df_model.groupby(cot_nhom)['LuongDuDoan'].mean().reset_index()
        kq['LuongDuDoan'] = kq['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
        kq = kq.rename(columns={
            cot_nhom: ten_moi,
            'LuongDuDoan': 'Thu nhập dự đoán'
        })
        return kq

    def ket_qua(self):
        print("\nDự đoán thu nhập trung bình theo ngành nghề:")
        print(self.tong_hop('DevType', 'Ngành nghề'))
        print("\nTheo hình thức làm việc:")
        print(self.tong_hop('RemoteWork', 'Hình thức làm việc'))
        print("\nTheo quốc gia:")
        print(self.tong_hop('Country', 'Quốc gia'))
        print("\nTheo trình độ học vấn:")
        print(self.tong_hop('EdLevel', 'Trình độ học vấn'))
        print("\nTheo quy mô công ty:")
        print(self.tong_hop('OrgSize', 'Quy mô công ty'))
        print("\nTheo số năm kinh nghiệm:")
        df_kq = self.df_model.groupby('WorkExp')['LuongDuDoan'].mean().reset_index()
        df_kq['LuongDuDoan'] = df_kq['LuongDuDoan'].round(0).apply(lambda x: f"{x:,.2f} USD")
        df_kq = df_kq.rename(columns={'WorkExp': 'Kinh nghiệm (năm)', 'LuongDuDoan': 'Thu nhập dự đoán'})
        print(df_kq)


predictor = Modeling_Thunhap(df)
predictor.tien_xu_ly()
predictor.train_model()
predictor.ket_qua()



Dự đoán thu nhập trung bình theo ngành nghề:
             Ngành nghề    Thu nhập dự đoán
0    Back-End Developer  232,937,713.00 USD
1    Embedded Developer   13,551,603.00 USD
2   Front-End Developer   70,959,556.00 USD
3  Full-Stack Developer   60,592,045.00 USD
4        Game Developer   12,151,425.00 USD
5               Manager   42,236,995.00 USD
6      Mobile Developer   61,824,001.00 USD
7                 Other   46,994,753.00 USD
8            Researcher   96,472,094.00 USD

Theo hình thức làm việc:
  Hình thức làm việc    Thu nhập dự đoán
0             Hybrid  115,194,422.00 USD
1          In-person  304,637,315.00 USD
2             Remote   35,067,764.00 USD

Theo quốc gia:
                                             Quốc gia      Thu nhập dự đoán
0                                           Argentina     15,136,418.00 USD
1                                           Australia     18,453,014.00 USD
2                                             Austria     17,282,240.00 USD
3   